# Notebook for building "distance matrix" for songs

We can think of song distance being a function of how many times songs appear together in playlists. That is, songs that appear in playlists together frequently have a smaller distance, and vice versa for those appearing together infrequently. 

In order to track distances we build a matrix $M$ where $M(i, j)$ is the distance between the $i$'th and $j$'th song. In order to save space, we represent this as a dictionary of dictionaries so that songs that do not ever appear together do not have to hold space in the representation. That is, the key of the first dictionary is a song, and its value is a dictionary of all other songs it appears in playists with and their corresponding distance. 

**NOTE:** Work in progress

In [60]:
import datetime
import json
import os
import sys
import time
from collections import OrderedDict 

import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

In [40]:
# top_songs = pd.Series(np.load("data/top_songs.npy", allow_pickle=True))
top_songs = set(np.load("data/top_songs.npy", allow_pickle=True))

In [41]:
top_songs

{'spotify:track:11zSZf9PeCHnGtWHdNkNHH',
 'spotify:track:2fvisKVcmhGrZdf9FPEhl4',
 'spotify:track:2kdpkYG1L8hARwR5EZofDL',
 'spotify:track:5iM0bUieH4msAenBJJoK4x',
 'spotify:track:2Sxg6TUyogtKWcAaRIwMVk',
 'spotify:track:514PMzqQ6fyrpRVVYXJS6S',
 'spotify:track:64nZJXDlu9Y0i4wnW9yO83',
 'spotify:track:5wVdKjASDMTyYSD1KAgtqs',
 'spotify:track:1bDnJAQ28P1k2mTSLiaip5',
 'spotify:track:6kRrRe66uS4QBuidjlBf1P',
 'spotify:track:6QVyQR4XcgJokOKDZnFkUV',
 'spotify:track:7e2aPfrwX7tCcC4wMDnsU4',
 'spotify:track:4cZX2I7NRGQ5S4cs3SRQ0B',
 'spotify:track:05R2BUnFy4KlkT4Ewg4Gnv',
 'spotify:track:6zFpeXenm0R9izFGwmPmQ8',
 'spotify:track:5YStYNF3SxYHqc5b0eubiR',
 'spotify:track:0E1NL6gkv5aQKGNjJfBE3A',
 'spotify:track:3l78vP424XB904B5MMmEdN',
 'spotify:track:7AqWd1WJxS1FgKMb4vyL3O',
 'spotify:track:1kv3HSZa4oLQkhk3VvGC0E',
 'spotify:track:7qGLqwVasl06Tu7W04OaCq',
 'spotify:track:7AR4urInDGgzBbbBplx31f',
 'spotify:track:1NxXjc3JfGJmQ6CvY1k7zw',
 'spotify:track:6MSihuO1ylprVatM3DwmRo',
 'spotify:track:

In [32]:
# """
# TODO: Change JSON to pkl 
# """
# Overlaps = OrderedDict({song: {song: 0} for song in top_songs})

# with open('data/overlaps.json', 'w') as outfile:
#     json.dump(Overlaps, outfile)

In [33]:
# start_idx = 0
# end_idx = 50
# total_slices = 50

In [34]:
# # TODO: Change JSON to pkl 
# Overlaps = json.load(open("data/overlaps.json", 'r'))

# slices = [f"data/Songs/{f}" for f in os.listdir('data/Songs') if f.endswith(".csv")]

# start_time = time.time()

# for i, slc in enumerate(slices[start_idx:end_idx]): 
#     slice_df = pd.read_csv(slc)
#     for playlist_df in [group for _, group in slice_df.groupby("pid")]:
#         tracks = playlist_df.track_uri
#         for track1 in tracks:
#             if track1 in top_songs:
#                 for track2 in tracks: 
#                     if track2 in top_songs: 
#                         if track2 in Overlaps[track1]:
#                             Overlaps[track1][track2] += 1
#                         else:
#                             Overlaps[track1][track2] = 1
#                 Overlaps[track1][track1] += 1

#     # TODO: Change JSON to pkl (and put this at the end of the block)
#     with open('data/overlaps.json', 'w') as outfile:
#         json.dump(Overlaps, outfile)
    
#     current_time = time.time()
#     elapsed_time = current_time - start_time
#     time_left = total_slices * elapsed_time / float(i+1) - elapsed_time
#     sys.stdout.write(
#         f"\rParsed slice {i+1}/{total_slices}. Time remaining: {str(datetime.timedelta(seconds=time_left))[:-7]}")
#     sys.stdout.flush()

Parsed slice 50/50. Time remaining: 0:02:50

In [55]:
list(Overlaps.items())[0]

('spotify:track:11zSZf9PeCHnGtWHdNkNHH',
 {'spotify:track:11zSZf9PeCHnGtWHdNkNHH': 2,
  'spotify:track:4NZg8df9BxJ6spPDnr6Z9T': 1,
  'spotify:track:0fTFmyy8AwWHiRoDgTh80w': 1,
  'spotify:track:6jgW3aGIRhVnZh5eLG0b3r': 1,
  'spotify:track:0lMwsRjgzmEhiv3anRRl7q': 1,
  'spotify:track:1wC0PdPIO8XABoPmz1PTWq': 1,
  'spotify:track:0HIO0Q2CKAueJIWBRKqKxV': 1,
  'spotify:track:1FjgsSAUEc2dDwk7SnuC6g': 1,
  'spotify:track:121GFU91AV4rrSN93sznhf': 1,
  'spotify:track:6Oyn1cIKVaaDTrsBAkFgTI': 1,
  'spotify:track:3usnkAX5lhr8R9oWjSmqZN': 1,
  'spotify:track:15b2Ft2XeuP7KqoPVSL3EC': 1,
  'spotify:track:09D7Il1PRMFjJqtZ9Fdwa3': 1,
  'spotify:track:1KoAx9SDjeA2SUHidrzMm5': 1,
  'spotify:track:51hSi1p7g2KdXg7fJA1Hgp': 1,
  'spotify:track:3MdvrasEZRZdvtlfAWLPZD': 1,
  'spotify:track:2vTdEBKuXAFi6T0IbRuDff': 1,
  'spotify:track:3UgSQu6WwrXfKKDq019IHE': 1,
  'spotify:track:3f9DeMtU3Sifmf5dqs2Ava': 1,
  'spotify:track:5HYwRSalrvZC7ZTT8BuDR7': 1,
  'spotify:track:2Da5ZQRk0BYuB0GRyrfIR1': 1,
  'spotify:tra

In [56]:
Distances = OrderedDict({song: {song: 0} for song in top_songs})
for song1, overlaps in list(Overlaps.items()):
    total_occurences = overlaps[song1]
    for song2, count in overlaps.items():
        if count:
            Distances[song1][song2] = 1 - count/total_occurences
    Distances[song1][song1] = 0

In [61]:
output = open('distances.pkl', 'wb')
pickle.dump(Distances, output)
output.close()